# Reading Data from TimeDB into Pandas DataFrames

This notebook demonstrates how to read data from TimeDB/PostgreSQL and work with it as pandas DataFrames.

## What you'll learn:
- Reading time series data into DataFrames
- Understanding flat vs overlapping query modes
- Reshaping data for analysis
- Filtering and querying data
- Converting back to wide format (pivot)


In [ ]:
import os
import uuid
import pandas as pd
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv

from timedb.db import create, insert, read

load_dotenv()

# Get database connection string from environment
conninfo = os.environ.get("TIMEDB_DSN") or os.environ.get("DATABASE_URL")
if not conninfo:
    raise ValueError("Set TIMEDB_DSN or DATABASE_URL environment variable")

# Set up IDs (use the same ones from the write notebook if continuing)
tenant_id = uuid.uuid4()
series_id = uuid.uuid4()

# Create schema and insert some sample data
print("Setting up sample data...")
create.create_schema(conninfo)

# Insert sample data with multiple value keys
base_time = datetime(2025, 1, 1, 0, 0, tzinfo=timezone.utc)
value_rows = []
for i in range(24):
    valid_time = base_time + timedelta(hours=i)
    value_rows.extend([
        (tenant_id, valid_time, series_id, "mean", 100.0 + i * 0.5),
        (tenant_id, valid_time, series_id, "quantile:0.1", 95.0 + i * 0.4),
        (tenant_id, valid_time, series_id, "quantile:0.9", 105.0 + i * 0.6),
    ])

insert.insert_run_with_values(
    conninfo,
    run_id=uuid.uuid4(),
    tenant_id=tenant_id,
    workflow_id="read-example",
    run_start_time=datetime.now(timezone.utc),
    run_finish_time=None,
    value_rows=value_rows,
)

print("✓ Sample data inserted")


## Example 1: Reading in Flat Mode

Flat mode returns the latest value for each (valid_time, value_key) combination. This is useful when you want the most recent forecast for each timestamp.


In [ ]:
# Read data in flat mode
df_flat = read.read_values_between(
    conninfo,
    tenant_id=tenant_id,
    start_valid=base_time,
    end_valid=base_time + timedelta(hours=24),
    mode="flat",
)

print("DataFrame shape:", df_flat.shape)
print("\nIndex structure:", df_flat.index.names)
print("\nFirst few rows:")
print(df_flat.head(10))


The DataFrame has a MultiIndex on `(valid_time, value_key)`. Let's reset the index to work with it more easily:


In [ ]:
# Reset index to make valid_time and value_key regular columns
df_flat_reset = df_flat.reset_index()

print("DataFrame with reset index:")
print(df_flat_reset.head(10))
print(f"\nShape: {df_flat_reset.shape}")
print(f"\nValue keys: {df_flat_reset['value_key'].unique()}")


## Converting to Wide Format (Pivot)

Often you'll want to convert the long-format DataFrame back to wide format with separate columns for each value_key:


In [ ]:
# Pivot to wide format: valid_time as index, value_key as columns
df_wide = df_flat_reset.pivot(
    index='valid_time',
    columns='value_key',
    values='value'
)

print("Wide format DataFrame:")
print(df_wide.head(10))
print(f"\nShape: {df_wide.shape}")
print(f"\nColumns: {df_wide.columns.tolist()}")


# Example: Calculate the range between quantiles
df_wide['range'] = df_wide['quantile:0.9'] - df_wide['quantile:0.1']

# Example: Plot the data
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 6))
df_wide[['mean', 'quantile:0.1', 'quantile:0.9']].plot(ax=ax)
ax.set_title('Time Series with Confidence Intervals')
ax.set_xlabel('Valid Time')
ax.set_ylabel('Value')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nBasic statistics:")
print(df_wide[['mean', 'quantile:0.1', 'quantile:0.9']].describe())


## Example 2: Reading in Overlapping Mode

Overlapping mode shows all forecast revisions for each timestamp. This is useful for backtesting and understanding how forecasts evolved over time.


In [ ]:
# First, let's insert a revised forecast to demonstrate overlapping mode
run_id_2 = uuid.uuid4()
known_time_2 = datetime.now(timezone.utc) + timedelta(hours=1)  # Later known_time

value_rows_2 = []
for i in range(24):
    valid_time = base_time + timedelta(hours=i)
    # Slightly different values for the revised forecast
    value_rows_2.extend([
        (tenant_id, valid_time, series_id, "mean", 101.0 + i * 0.5),  # Slightly higher
        (tenant_id, valid_time, series_id, "quantile:0.1", 96.0 + i * 0.4),
        (tenant_id, valid_time, series_id, "quantile:0.9", 106.0 + i * 0.6),
    ])

insert.insert_run_with_values(
    conninfo,
    run_id=run_id_2,
    tenant_id=tenant_id,
    workflow_id="read-example-revised",
    run_start_time=known_time_2,
    known_time=known_time_2,
    value_rows=value_rows_2,
)

print("✓ Inserted revised forecast")


In [ ]:
# Read in overlapping mode
df_overlapping = read.read_values_between(
    conninfo,
    tenant_id=tenant_id,
    start_valid=base_time,
    end_valid=base_time + timedelta(hours=24),
    mode="overlapping",
)

print("DataFrame shape:", df_overlapping.shape)
print("\nIndex structure:", df_overlapping.index.names)
print("\nFirst few rows:")
print(df_overlapping.head(15))


The overlapping mode DataFrame has a MultiIndex on `(known_time, valid_time, value_key)`. Let's reset it:


In [ ]:
# Reset index for easier manipulation
df_overlapping_reset = df_overlapping.reset_index()

print("DataFrame with reset index:")
print(df_overlapping_reset.head(20))
print(f"\nUnique known_time values: {df_overlapping_reset['known_time'].nunique()}")
print(f"Unique valid_time values: {df_overlapping_reset['valid_time'].nunique()}")


Let's see how the forecast for a specific timestamp evolved over time:


In [ ]:
# Filter for a specific valid_time to see how the forecast evolved
specific_time = base_time + timedelta(hours=12)
df_specific = df_overlapping_reset[
    (df_overlapping_reset['valid_time'] == specific_time) &
    (df_overlapping_reset['value_key'] == 'mean')
].sort_values('known_time')

print(f"Forecast evolution for valid_time={specific_time}:")
print(df_specific[['known_time', 'value']])


## Example 3: Filtering and Querying

You can filter data by time ranges and specific value keys:


In [ ]:
# Read a specific time range
start_time = base_time + timedelta(hours=6)
end_time = base_time + timedelta(hours=18)

df_filtered = read.read_values_between(
    conninfo,
    tenant_id=tenant_id,
    start_valid=start_time,
    end_valid=end_time,
    mode="flat",
)

print(f"Filtered DataFrame shape: {df_filtered.shape}")
print(f"Time range: {df_filtered.reset_index()['valid_time'].min()} to {df_filtered.reset_index()['valid_time'].max()}")
print("\nFirst few rows:")
print(df_filtered.head())


In [ ]:
# Filter for specific value keys
df_filtered_reset = df_filtered.reset_index()
df_mean_only = df_filtered_reset[df_filtered_reset['value_key'] == 'mean']

print("Mean values only:")
print(df_mean_only.head(10))


## Helper Function: Read and Convert to Wide Format

Here's a helper function that reads data and automatically converts it to wide format:
        

In [ ]:
def read_timedb_to_wide_dataframe(
    conninfo: str,
    tenant_id: uuid.UUID,
    start_valid: datetime = None,
    end_valid: datetime = None,
    value_keys: list = None,
) -> pd.DataFrame:
    """
    Read data from TimeDB and return it as a wide-format DataFrame.
    
    Args:
        conninfo: Database connection string
        tenant_id: Tenant UUID
        start_valid: Start of valid time range (optional)
        end_valid: End of valid time range (optional)
        value_keys: List of value keys to include (None for all)
    
    Returns:
        DataFrame with valid_time as index and value_keys as columns
    """
    # Read in flat mode
    df = read.read_values_between(
        conninfo,
        tenant_id=tenant_id,
        start_valid=start_valid,
        end_valid=end_valid,
        mode="flat",
    )
    
    # Reset index
    df_reset = df.reset_index()
    
    # Filter by value keys if specified
    if value_keys:
        df_reset = df_reset[df_reset['value_key'].isin(value_keys)]
    
    # Pivot to wide format
    df_wide = df_reset.pivot(
        index='valid_time',
        columns='value_key',
        values='value'
    )
    
    return df_wide


# Example usage
df_wide_helper = read_timedb_to_wide_dataframe(
    conninfo,
    tenant_id=tenant_id,
    start_valid=base_time,
    end_valid=base_time + timedelta(hours=24),
    value_keys=['mean', 'quantile:0.1', 'quantile:0.9']
)

print("Wide format DataFrame from helper:")
print(df_wide_helper.head(10))
print(f"\nShape: {df_wide_helper.shape}")


## Summary

You've learned how to:
1. ✅ Read data from TimeDB into pandas DataFrames
2. ✅ Understand flat vs overlapping query modes
3. ✅ Convert long-format to wide-format DataFrames (pivot)
4. ✅ Filter and query data by time ranges and value keys
5. ✅ Analyze forecast revisions over time
6. ✅ Use helper functions to simplify common operations

**Key Points:**
- **Flat mode**: Returns latest value per (valid_time, value_key) - use for current forecasts
- **Overlapping mode**: Returns all revisions - use for backtesting and analysis
- DataFrames use MultiIndex - reset index for easier manipulation
- Use `pivot()` to convert from long to wide format
- All datetimes are timezone-aware

**Next Steps:**
- Combine with `notebook_01_write_dataframe.ipynb` for a complete workflow
- See `notebook_03_complete_workflow.ipynb` for a real-world example
